# Intro 
Here we are going to to analyze the Reiner Zonneveld's time warp perfoamcne. 

# Load the data
we will consider this part of the performance: 
01:31:00 DJ Promo - My underground madness
01:33:03 Age of Love - Age of Love [some RMX or sped up version]
01:36:00 Reinier Zonneveld & Angerfist - Fist on Acid
01:41:00 Miro - Shining (Reinier Zonneveld RMX)

# Analysis. 


In [ ]:
audio_path = "/Users/nimamanaf/Desktop/Music/Reiner Zonneveld - Time Warp 2023/Reinier Zonneveld - Time Warp 2023.wav"


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
import ipywidgets as widgets
from IPython.display import display, Audio, clear_output
import plotly.graph_objects as go

class DJAnalysis:
    def __init__(self, audio, sr=None, info=None):
        if info is not None:
            self.song_starts = self.convert_text_to_song_info(info)
            self.song_starts_seconds = list(self.song_starts.keys())
            self.start = self.song_starts_seconds[0]
            self.duration = self.song_starts_seconds[-1] - self.start
        if isinstance(audio, str):
            self.y, self.sr = librosa.load(audio, duration=self.duration, offset=self.start)
        else:
            self.y = audio
            self.sr = sr
        
        self.tempo = None
        self.key = None
        self.energy = None

    @staticmethod
    def convert_to_seconds(time):
        """
        Converts a time string in the format "hh:mm:ss" to seconds.
        """
        time = time.split(':')
        return int(time[0]) * 3600 + int(time[1]) * 60 + int(time[2])
    
    @staticmethod
    def convert_text_to_song_info(text):
        song_info = {}
        lines = text.split('\n')
        
        for line in lines:
            parts = line.split(' ')
            if len(parts) >= 3:
                time_parts = parts[0].split(':')
                if len(time_parts) == 3:
                    hours = int(time_parts[0])
                    minutes = int(time_parts[1])
                    seconds = int(time_parts[2])
                    start_time_seconds = hours * 3600 + minutes * 60 + seconds
                    song_name = ' '.join(parts[2:])
                    song_info[start_time_seconds] = song_name
        
        return song_info
    
    def get_sliding_tempo(self, window_size=30, window_step=5):
        """
        This function calculates the tempo every 10 seconds with an sliding window of 5 seconds.
        :param y: audio time series
        :param sr: sampling rate of `y`
        :param window_size: size of the window in seconds
        :param window_step: step of the window in seconds
        :return: a list of tuples with the tempo and the time in seconds
        """
        y, sr = self.y, self.sr
        # get the total time of the track in seconds
        total_time = librosa.get_duration(y=y, sr=sr)
        # calculate the number of windows 
        num_windows = int(np.ceil((total_time - window_size) / window_step))
        # initialize the list of tuples
        tempo_list = []
        # iterate over the windows
        for i in range(num_windows):
            # get the start and end time of the window
            start = i * window_step
            end = start + window_size
            # calculate the tempo
            tempo = librosa.beat.beat_track(y=y[start*sr:end*sr], sr=sr)[0]
            # append the tuple
            tempo_list.append((tempo, start))
        return tempo_list
    
    def get_sliding_energy(self, window_size=30, window_step=5):
        # Similar to get_sliding_tempo, but for RMS energy
        total_time = librosa.get_duration(y=self.y, sr=self.sr)
        num_windows = int(np.ceil((total_time - window_size) / window_step))
        energy_list = []
        for i in range(num_windows):
            start = i * window_step
            end = start + window_size
            rms_energy = np.mean(librosa.feature.rms(y=self.y[start*self.sr:end*self.sr]))
            energy_list.append((rms_energy, start))
        return energy_list

    def get_sliding_key(self, window_size=30, window_step=5):
        total_time = librosa.get_duration(y=self.y, sr=self.sr)
        num_windows = int(np.ceil((total_time - window_size) / window_step))
        key_list = []
        for i in range(num_windows):
            start = i * window_step
            end = start + window_size
            chroma = librosa.feature.chroma_cqt(y=self.y[start*self.sr:end*self.sr], sr=self.sr)
            key = np.argmax(np.sum(chroma, axis=1))
            key_list.append((key, start))
        return key_list
    
    def get_sliding_spectral_centroid(self, window_size=30, window_step=5):
        total_time = librosa.get_duration(y=self.y, sr=self.sr)
        num_windows = int(np.ceil((total_time - window_size) / window_step))
        centroid_list = []
        for i in range(num_windows):
            start = i * window_step
            end = start + window_size
            centroid = np.mean(librosa.feature.spectral_centroid(y=self.y[start*self.sr:end*self.sr], sr=self.sr))
            centroid_list.append((centroid, start))
        return centroid_list

    def get_sliding_spectral_bandwidth(self, window_size=30, window_step=5):
        total_time = librosa.get_duration(y=self.y, sr=self.sr)
        num_windows = int(np.ceil((total_time - window_size) / window_step))
        bandwidth_list = []
        for i in range(num_windows):
            start = i * window_step
            end = start + window_size
            bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=self.y[start*self.sr:end*self.sr], sr=self.sr))
            bandwidth_list.append((bandwidth, start))
        return bandwidth_list

    def get_sliding_hnr(self, window_size=30, window_step=5):
        # HNR might be more computationally intensive, so this method might be slower
        total_time = librosa.get_duration(y=self.y, sr=self.sr)
        num_windows = int(np.ceil((total_time - window_size) / window_step))
        hnr_list = []
        for i in range(num_windows):
            start = i * window_step
            end = start + window_size
            hnr_value = librosa.effects.harmonic(self.y[start*self.sr:end*self.sr])[0]
            hnr_list.append((hnr_value, start))
        return hnr_list

    def get_sliding_zero_crossing_rate(self, window_size=30, window_step=5):
        total_time = librosa.get_duration(y=self.y, sr=self.sr)
        num_windows = int(np.ceil((total_time - window_size) / window_step))
        zcr_list = []
        for i in range(num_windows):
            start = i * window_step
            end = start + window_size
            zcr = np.mean(librosa.feature.zero_crossing_rate(self.y[start*self.sr:end*self.sr]))
            zcr_list.append((zcr, start))
        return zcr_list
    
    def get_sliding_spectral_contrast(self, window_size=30, window_step=5):
        total_time = librosa.get_duration(y=self.y, sr=self.sr)
        num_windows = int(np.ceil((total_time - window_size) / window_step))
        contrast_list = []
        for i in range(num_windows):
            start = i * window_step
            end = start + window_size
            contrast = np.mean(librosa.feature.spectral_contrast(y=self.y[start*self.sr:end*self.sr], sr=self.sr), axis=1)
            contrast_list.append((contrast, start))
        return contrast_list

    def get_band_energy_ratios(self, window_size=30, window_step=5):
        total_time = librosa.get_duration(y=self.y, sr=self.sr)
        num_windows = int(np.ceil((total_time - window_size) / window_step))
        bands_list = []
        for i in range(num_windows):
            start = i * window_step
            end = start + window_size
            D = np.abs(librosa.stft(self.y[start*self.sr:end*self.sr]))
            band_energy = [np.mean(D[i]) for i in [range(0, 22), range(22, 45), range(45, 102)]]
            bands_list.append((band_energy, start))
        return bands_list

    def extract_segment_from_time(self, start_time, duration=None):
        """
        Extracts a segment of the audio based on a start time and duration (in seconds). If the duration is not provided, the segment is extracted until the end of the audio.
        """
        if duration is None:
            duration = len(self.y) / self.sr - start_time
        return self.y[start_time*self.sr:(start_time+duration)*self.sr]

    def get_segments(self, ):
        """
        Extracts segments of the audio based on the song start times.
        """
        segments = []
        for idx, start_time in enumerate(self.song_starts_seconds):
            if idx<len(self.song_starts_seconds)-1:
                duration = self.song_starts_seconds[idx+1] - start_time
            else:
                duration = None
            segments.append(self.extract_segment_from_time(start_time, duration))
 
    def extract_tempo(self):
        onset_env = librosa.onset.onset_strength(y=self.y, sr=self.sr)
        self.tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=self.sr)
        return self.tempo

    def extract_key(self):
        chroma = librosa.feature.chroma_cqt(y=self.y, sr=self.sr)
        self.key = librosa.key(chroma)[0]  # Only the key, not the scale
        return self.key

    def extract_energy(self):
        rms_energy = librosa.feature.rms(y=self.y)
        self.energy = rms_energy[0]
        return self.energy

    def visualize_waveform(self, start_time=None, duration=None):
        """
        Visualizes the waveform of a segment specified by start_time and duration.
        If start_time is None, it visualizes from the start of the audio.
        If duration is None, it visualizes until the end of the audio.
        """

        if start_time is None:
            start_time = 0

        # Calculate start and end sample
        start_sample = int(start_time * self.sr)

        if duration is None:
            end_sample = len(self.y)
        else:
            end_sample = start_sample + int(duration * self.sr)

        # Ensure we don't exceed the audio's length
        end_sample = min(end_sample, len(self.y))

        # Extract segment
        y_segment = self.y[start_sample:end_sample]

        # Time axis for segment
        time_segment = np.linspace(start_time, start_time + (end_sample - start_sample) / self.sr, len(y_segment))

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=time_segment, y=y_segment, mode='lines', name='Waveform'))

        fig.update_layout(title='Waveform Segment', xaxis_title='Time (s)', yaxis_title='Amplitude')
        fig.show()

    def visualize_features(self, features=None, window_size=30, window_step=5):

        self.visualize_waveform()

        feature_methods = {
            "tempo": self.get_sliding_tempo,
            "energy": self.get_sliding_energy,
            "key": self.get_sliding_key,
            "spectral centroid": self.get_sliding_spectral_centroid,
            "spectral bandwidth": self.get_sliding_spectral_bandwidth,
            "hnr": self.get_sliding_hnr,
            "zero crossing rate": self.get_sliding_zero_crossing_rate,
            "spectral contrast": self.get_sliding_spectral_contrast,
            "band energy ratios": self.get_band_energy_ratios
        }
        
        if features is None:
            features = feature_methods.keys()
        
        for feature in features:
            print(f"Extracting {feature}...")
            data_list = feature_methods[feature](window_size, window_step)
            times = [t[1] for t in data_list]
            values = [t[0] for t in data_list]
            
            fig = go.Figure(data=[go.Scatter(x=times, y=values, mode='lines', name=feature.capitalize())])
            fig.update_layout(title=f"{feature.capitalize()}", xaxis_title='Time (s)', yaxis_title=feature.capitalize())
            fig.show()
    

In [ ]:

info = '''
01:31:00 DJ Promo - My underground madness
01:33:03 Age of Love - Age of Love [some RMX or sped up version]
01:36:00 Reinier Zonneveld & Angerfist - Fist on Acid
01:41:00 Miro - Shining (Reinier Zonneveld RMX)
'''
dj = DJAnalysis(audio_path, sr=None, info=info)


In [ ]:
dj.visualize_features(features=None, window_size=30, window_step=20)

In [ ]:
dj.visualize_waveform(start_time=120, duration=120)

In [ ]:
# play the audio making the widget size cover the whole screen
display(Audio(audio_path, autoplay=True, rate=dj.sr), display_id='audio')   

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

In [ ]:

# Load the audio file
song = AudioSegment.from_wav("/Users/nimamanaf/Library/CloudStorage/GoogleDrive-ndizbin14@ku.edu.tr/My Drive/Techno/technob/docs/examples/cse.WAV")

# Play the audio file
play(song)
